<a href="https://colab.research.google.com/github/viswambhar-yasa/Where_r_Bees/blob/main/where_r_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8 MB 22 kB/s 
     |████████████████████████████████| 6.6 MB 21.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+cu113
    Uninstalling torchvision-0.13.0+cu113:
      Successfully uninstalled torchvision-0.13.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.0 requires torch==1.12.0, but you have torch 1.5.0+cu101 which is incompatible.
torchaudio 0.12.0+cu113 requires torch==1.12.0, but you have torch 1.5.0+cu101

In [2]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2 MB 787 kB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=9548e784ba5263804f6f49b804412abde699f5818491de1a5a8fc187afcc5bfa
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31549 sha256=f5210f32a04a111fee34899df543ab3b266539f4547bb877090c4ea4433e4581
  Stored in directory: /root/.cache/pip/wheels/aa/cc/ed/ca4e88beef656b01c84b9185196513ef2faf74a5a379b043a7
Successfully built fvcore iopath


In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets
import ipywidgets as widgets
import os,subprocess
from IPython.display import display,clear_output

In [4]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
import random
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

In [5]:
def mount_drive_button():
  button = widgets.Button(description='Mount G-Drive')
  display(button)
  def on_button_clicked(b):
      from google.colab import drive
      drive.mount('/content/drive', force_remount=False)
      with drive_output:
        clear_output()
        print('drive mounted')
  button.on_click(on_button_clicked)
  drive_output=widgets.Output()
  display(drive_output)


def path_load():
    file_location = str(get_file_path.output_variable.value)
    if file_location!='':
      #path_load.path_choice = path/file_location
      path_load.path_choice = file_location
      print('\n selected path \n')
      print(path_load.path_choice)
      if path_load.path_choice[-3:]=='zip':
          print('Found Zip file, do you want to unzip it')
          print('Data will be extracted to: /content/data/')
          unzipbutton = widgets.Button(description='Unzip')
          display(unzipbutton)
          def unzip_file(b):
              cmd="unzip "+path_load.path_choice +" -d /content/data/"
              env = os.environ.copy()
              subprocess.run(cmd, shell=True, env=env)
              path_load.path_choice='/content/data/'
              get_file_path.output_variable='/content/data/'
          unzipbutton.on_click(unzip_file)  
    else:
      print('\n Default path selected')
      file_location = str('/content/data/')
      #path_load.path_choice = path/file_location
      path_load.path_choice = file_location
      get_file_path.output_variable='/content/data/'
      print(get_file_path.output_variable)


def get_file_path():
  print('select the path of the files')
  button_choice = widgets.Button(description="Load Path")
  get_file_path.output_variable = widgets.Text()
  display(get_file_path.output_variable)
  def on_button_clicked(b):
    with fileout:
      clear_output()
      path_load()
  display(button_choice)
  button_choice.on_click(on_button_clicked)
  fileout = widgets.Output()
  display(fileout)

In [6]:
## system information
def get_system_info():
  import os
  import tensorflow as tf
  print ('>> System info <<\n')

  import torch, torchvision
  print('Pytorch version:',torch.__version__)
  print('\n GPU avaliablity:',torch.cuda.is_available())
  
  def install_packages(b):
    with ipout:
      clear_output()
      print('Installing packages from requirement text')
      cmd="pip install -r requirements.txt"
      env = os.environ.copy()
      subprocess.run(cmd, shell=True, env=env)

  inbutton = widgets.Button(description='Install Packages')
  display(inbutton)
  inbutton.on_click(install_packages)
  ipout = widgets.Output()
  display(ipout)

In [7]:
def data_visualization():
  
  print('Check if the Dataset exist else create them')
  databutton = widgets.Button(description='Datasets')
  display(databutton)
  subfolder=[]
  ds=None
  def check_datasets(b):
    with vis_out:
      clear_output()
      print('Building Datsets \n')
      if str(get_file_path.output_variable)!='':
        sub_dirs = os.listdir(str(get_file_path.output_variable))
        if 'train' in sub_dirs:
          subfolder.append('train')
          try:
            register_coco_instances("my_dataset_train", {}, str(get_file_path.output_variable)+"train/_annotations.coco.json", str(get_file_path.output_variable)+"train")
            print('Training Dataset Built \n')
          except:
            print('Training Dataset already exists')
        if 'test' in sub_dirs:
          subfolder.append('test')
          try:
            register_coco_instances("my_dataset_test", {}, str(get_file_path.output_variable)+"test/_annotations.coco.json", str(get_file_path.output_variable)+"test")
            print('Test Dataset Built \n')
          except:
            print('Test Dataset already exists')
        if 'valid' in sub_dirs:
          subfolder.append('valid')
          try:
            register_coco_instances("my_dataset_valid", {}, str(get_file_path.output_variable)+"valid/_annotations.coco.json", str(get_file_path.output_variable)+"valid")
            print('Validation Dataset Built \n')
          except:
            print('Validation Dataset already exists')
        check_datasets.datas = widgets.ToggleButtons(
                                options=subfolder,
                                description='Choose',
                                value='train',
                                disabled=False,
                                button_style='info',
                                tooltips=[''])
        display(check_datasets.datas)
        ds=check_datasets.datas.value
  def visualize_data(b):
      with im_out:
        clear_output()
        if check_datasets.datas.value is None:
          print('select the dataset for visualization')
        else:
          print("Visualization of "+check_datasets.datas.value+ " Datasets")
          my_dataset_train_metadata = MetadataCatalog.get("my_dataset_"+check_datasets.datas.value)
          dataset_dicts = DatasetCatalog.get("my_dataset_"+check_datasets.datas.value)  
          for d in random.sample(dataset_dicts, 3):
            img = cv2.imread(d["file_name"])
            visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
            vis = visualizer.draw_dataset_dict(d)
            cv2_imshow(vis.get_image()[:, :, ::-1])
  
  databutton.on_click(check_datasets)
  vis_out=widgets.Output()
  display(vis_out) 
  visbutton = widgets.Button(description='visualize data')
  display(visbutton)
  visbutton.on_click(visualize_data) 
  im_out=widgets.Output()
  display(im_out) 

In [8]:

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [99]:
def train_model(cfg):
  trainer = CocoTrainer(cfg)
  trainer.resume_or_load(resume=False)
  trainer.train()
  pass

def model():
  model.datas = widgets.ToggleButtons(
        options=['FRCNN', 'Others'],
        description='DL model',
        value='FRCNN',
        disabled=False,
        button_style='info',
        tooltips=['']
    )
  display(model.datas)
  print(model.datas.value)
  tnw= widgets.Button(description='Num of worker', disabled=True)
  model.nw = widgets.ToggleButtons(value='4', options=['4','8', '16', '32', '64'], description='', button_style='warning')
  tbs = widgets.Button(description='Batch Size', disabled=True)
  model.bs = widgets.ToggleButtons(value='4', options=['4','8', '16', '32', '64'], description='', button_style='warning')
  tlr= widgets.Button(description='Learning rate', disabled=True)
  model.lr = widgets.FloatSlider(value=0.001,min=0,max=1,step=0.001, description='lr',
                                     orientation='horizontal', disabled=False,readout_format='.3f')
  tni = widgets.Button(description='Images per batch', disabled=True)
  model.ni = widgets.ToggleButtons(value='16', options=['16', '32', '64','128'], description='', button_style='warning')
  tgamma= widgets.Button(description='Gamma', disabled=True)
  model.gamma = widgets.FloatSlider(value=0.05,min=0,max=1,step=0.01, description='gm',
                                     orientation='horizontal', disabled=False)
  tti= widgets.Button(description='Warming Training epochs', disabled=True)
  model.ti = widgets.IntSlider(value=100,min=100,max=10000,step=100, description='warm epochs',
                                     orientation='horizontal', disabled=False)
  tmi= widgets.Button(description='Max Training epochs', disabled=True)
  model.mi = widgets.IntSlider(value=500,min=100,max=10000,step=100, description='Max epochs',
                                     orientation='horizontal', disabled=False)
  tnc= widgets.Button(description='Number of classes', disabled=True)
  model.nc = widgets.IntSlider(value=4,min=1,max=100,step=1, description='classes',
                                     orientation='horizontal', disabled=False)
  
  it = [tnw, model.nw]
  it2 = [tbs, model.bs]
  it3 = [tlr, model.lr]
  it4 = [tni, model.ni]
  it5 = [tgamma, model.gamma]
  it6 = [tti, model.ti]
  it7 =[tmi,model.mi]
  it8 =[tnc,model.nc]
  # it52 = [to, model.imgbth]
  # it6 = [tf, model.aug]
  il = widgets.HBox(it)
  ij = widgets.HBox(it2)
  ik = widgets.HBox(it3)
  ie = widgets.HBox(it4)
  iw = widgets.HBox(it5)
  # ip = widgets.HBox(it52)
  ip = widgets.HBox(it6)
  iq = widgets.HBox(it7)
  iu = widgets.HBox(it8)
  ir = widgets.VBox([il, ij, ik,ie,iw,ip,iq,iu])# ie, spj, iw, ip, iq])
  display(ir)
  def build_model(b):
      global cfg
      cfg = get_cfg()
      if model.datas.value=='FRCNN':
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
        cfg.DATASETS.TRAIN = ("my_dataset_train",)
        cfg.DATASETS.TEST = ("my_dataset_valid",)
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml") 
      else:
        print('selected default network')
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
        cfg.DATASETS.TRAIN = ("my_dataset_train",)
        cfg.DATASETS.TEST = ("my_dataset_valid",)
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml") 
      cfg.DATALOADER.NUM_WORKERS = int(str(model.nw.value))
      cfg.SOLVER.IMS_PER_BATCH = int(str(model.bs.value))
      cfg.SOLVER.BASE_LR = float(str(model.lr.value))


      cfg.SOLVER.WARMUP_ITERS = int(str(model.ti.value))
      cfg.SOLVER.MAX_ITER = int(str(model.mi.value)) #adjust up if val mAP is still rising, adjust down if overfit
      cfg.SOLVER.STEPS = (int(str(model.ti.value)), int(str(model.mi.value)))
      cfg.SOLVER.GAMMA = float(str(model.gamma.value))




      cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = int(str(model.ni.value))
      cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4 #your number of classes + 1

      cfg.TEST.EVAL_PERIOD = 500
      with tmt_out:
        clear_output()
        print('Model built based on the input hyperparameters')

  def train_model(p):
    with trn_out:
        clear_output()
        trainer = CocoTrainer(cfg)
        trainer.resume_or_load(resume=False)
        trainer.train()
  
  def tensorflowboard(b):
    with tb_out:
      clear_output()
      %load_ext tensorboard
      %tensorboard --logdir output
  tb_button = widgets.Button(description='learning curves', button_style='success')
  tb_button.on_click(tensorflowboard)
  tmt_button = widgets.Button(description='Build model', button_style='success')
  tmt_button.on_click(build_model)
  
  trn_button = widgets.Button(description='Train model', button_style='success')
  trn_button.on_click(train_model)
  display(tmt_button)
  tmt_out = widgets.Output()
  display(tmt_out)
  display(trn_button)
  trn_out = widgets.Output()
  display(trn_out)
  print('evaluation of the learning curves in TensorBoard')
  display(tb_button) 
  tb_out = widgets.Output()
  display(tb_out)


In [100]:
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

In [133]:
import json
import pandas as pd
from detectron2.utils.visualizer import ColorMode
import glob

def evaluation():
  def evaluate(b):
    with m_out:
      clear_output()
      print('Evaluation metrics on test data')
      try:
        trainer1 = CocoTrainer(cfg)
        cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85  
        predictor = DefaultPredictor(cfg)
        evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
        val_loader = build_detection_test_loader(cfg, "my_dataset_test")
        inference_on_dataset(trainer1.model, val_loader, evaluator)
      except:
        print('Please build the model before evaluation')
      # if os.path.exists(file_path):
      #     json_file= open(file_path)
      #     metrics=json.load(json_file)
      #     metrics_table=pd.DataFrame.from_dict(metrics)
      #     print(metrics)
  def predict(b):
    with p_out:
      clear_output()
      print('Where are BEES?')
      cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/output/model_final.pth")
      cfg.DATASETS.TEST = ("my_dataset_test", )
      cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
      predictor = DefaultPredictor(cfg)
      test_metadata = MetadataCatalog.get("my_dataset_test")
      print('Predicting on test data')
      for imageName in glob.glob('/content/data/test/*jpg'):
        im = cv2.imread(imageName)
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                      metadata=test_metadata, 
                      scale=0.8
                       )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        cv2_imshow(out.get_image()[:, :, ::-1])
    
  metrics_button = widgets.Button(description='Evaluate', button_style='success')
  
  metrics_button.on_click(evaluate)
  display(metrics_button)
  m_out = widgets.Output()
  display(m_out)
  predict_button = widgets.Button(description='Predict', button_style='success')
  display(predict_button)
  predict_button.on_click(predict)
  p_out = widgets.Output()
  display(p_out)

In [134]:
!pip freeze>requirements.txt

In [135]:
def where_r_B():
  from google.colab import widgets
  from google.colab import output
  import gc
  gc.collect()
  t=widgets.TabBar(['Drive','System Info','Data','Build and Training','Evaluation and prediction'])
  with t.output_to('Drive', select=False):  
    t.clear_tab()
    print('To load Google Drive to colab, please press the below button')
    mount_drive_button()
    get_file_path()
  with t.output_to('Data', select=False):
      t.clear_tab()
      data_visualization()
  with t.output_to('System Info', select=False):
    get_system_info()
  with t.output_to('Build and Training', select=False):
    model()
  with t.output_to('Evaluation and prediction', select=False):
    evaluation()

In [137]:
where_r_B()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

Button(button_style='success', description='Evaluate', style=ButtonStyle())

Output()

Button(button_style='success', description='Predict', style=ButtonStyle())

Output()

<IPython.core.display.Javascript object>